In [1]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [2]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2023-10-18 16:15:04,368 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-18 16:15:04,369 INFO sqlalchemy.engine.Engine select 'hello world'
2023-10-18 16:15:04,371 INFO sqlalchemy.engine.Engine [generated in 0.00289s] ()
[('hello world',)]
2023-10-18 16:15:04,373 INFO sqlalchemy.engine.Engine ROLLBACK


In [3]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2023-10-18 16:15:04,387 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-18 16:15:04,388 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-10-18 16:15:04,389 INFO sqlalchemy.engine.Engine [generated in 0.00240s] ()
2023-10-18 16:15:04,392 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-10-18 16:15:04,393 INFO sqlalchemy.engine.Engine [generated in 0.00117s] [(1, 1), (2, 4)]
2023-10-18 16:15:04,395 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2023-10-18 16:15:04,409 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-18 16:15:04,410 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-10-18 16:15:04,412 INFO sqlalchemy.engine.Engine [cached since 0.02023s ago] [(6, 8), (9, 10)]
2023-10-18 16:15:04,414 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-10-18 16:15:04,426 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-18 16:15:04,428 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-10-18 16:15:04,430 INFO sqlalchemy.engine.Engine [generated in 0.00345s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-10-18 16:15:04,432 INFO sqlalchemy.engine.Engine ROLLBACK


In [6]:
with engine.connect() as conn:
    result = conn.execute(text("select x, y from some_table"))


    for row in result:
        y = row.y

    # illustrate use with Python f-strings
        print(f"Row: {row.x} {y}")

2023-10-18 16:15:04,446 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-18 16:15:04,447 INFO sqlalchemy.engine.Engine select x, y from some_table
2023-10-18 16:15:04,449 INFO sqlalchemy.engine.Engine [generated in 0.00308s] ()
Row: 1 1
Row: 2 4
Row: 6 8
Row: 9 10
2023-10-18 16:15:04,450 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
with engine.connect() as conn:
    result = conn.execute(text("select x, y from some_table"))

    for dict_row in result.mappings():
        x = dict_row["x"]
        y = dict_row["y"]
        print(f"x: {x}  y: {y}")

2023-10-18 16:15:04,464 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-18 16:15:04,465 INFO sqlalchemy.engine.Engine select x, y from some_table
2023-10-18 16:15:04,466 INFO sqlalchemy.engine.Engine [cached since 0.02076s ago] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-10-18 16:15:04,467 INFO sqlalchemy.engine.Engine ROLLBACK


In [8]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-10-18 16:15:04,481 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-18 16:15:04,482 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2023-10-18 16:15:04,483 INFO sqlalchemy.engine.Engine [generated in 0.00281s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-10-18 16:15:04,486 INFO sqlalchemy.engine.Engine ROLLBACK


In [9]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

2023-10-18 16:15:04,499 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-18 16:15:04,500 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-10-18 16:15:04,501 INFO sqlalchemy.engine.Engine [cached since 0.1099s ago] [(11, 12), (13, 14)]
2023-10-18 16:15:04,503 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-10-18 16:15:04,721 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-18 16:15:04,723 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2023-10-18 16:15:04,725 INFO sqlalchemy.engine.Engine [generated in 0.00213s] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2023-10-18 16:15:04,727 INFO sqlalchemy.engine.Engine ROLLBACK


In [16]:
from sqlalchemy import MetaData
metadata_obj = MetaData()
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)
print(f"Table: {user_table.name}")
print(f"Columns: {user_table.c.keys()}")
print(f"Primary Keys: {user_table.primary_key}")

Table: user_account
Columns: ['id', 'name', 'fullname']
Primary Keys: PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))


In [17]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

In [18]:
metadata_obj.create_all(engine)

2023-10-19 02:10:11,471 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-19 02:10:11,472 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-10-19 02:10:11,473 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-19 02:10:11,475 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-10-19 02:10:11,476 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-19 02:10:11,478 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-10-19 02:10:11,480 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-19 02:10:11,481 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2023-10-19 02:10:11,482 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-19 02:10:11,485 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2023-10-19 02:10:11,486 INFO sqlalchemy.engine.Engine [no key 0.00085s] ()
2023-10-19 02:10:11,490 INFO sqlalchemy.engine.Engine 
C

In [19]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

In [20]:
Base.metadata

MetaData()

In [21]:
Base.registry

In [22]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2023-10-19 23:20:08,331 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-19 23:20:08,333 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2023-10-19 23:20:08,336 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-19 23:20:08,341 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2023-10-19 23:20:08,343 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2023-10-19 23:20:08,347 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2023-10-19 23:20:08,349 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-19 23:20:08,351 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2023-10-19 23:20:08,353 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-19 23:20:08,354 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type i

In [25]:
from sqlalchemy import insert
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
print(stmt)


INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [27]:
compiled = stmt.compile()
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

In [29]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()
result.inserted_primary_key

2023-10-19 23:49:34,524 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-19 23:49:34,527 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-10-19 23:49:34,528 INFO sqlalchemy.engine.Engine [cached since 20.34s ago] ('spongebob', 'Spongebob Squarepants')
2023-10-19 23:49:34,531 INFO sqlalchemy.engine.Engine COMMIT


(2,)

In [30]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2023-10-19 23:54:59,618 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-19 23:54:59,619 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-10-19 23:54:59,620 INFO sqlalchemy.engine.Engine [generated in 0.00323s] [('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star')]
2023-10-19 23:54:59,621 INFO sqlalchemy.engine.Engine COMMIT


In [31]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table))
    conn.commit()

2023-10-19 23:55:43,340 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-19 23:55:43,342 INFO sqlalchemy.engine.Engine INSERT INTO user_account DEFAULT VALUES
2023-10-19 23:55:43,344 INFO sqlalchemy.engine.Engine [generated in 0.00430s] ()
2023-10-19 23:55:43,349 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
insert_stmt = insert(address_table).returning(
    address_table.c.id, address_table.c.email_address
)
print(insert_stmt)

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


In [34]:
from sqlalchemy import select
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


In [35]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [36]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2023-10-20 00:14:40,033 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-20 00:14:40,035 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2023-10-20 00:14:40,037 INFO sqlalchemy.engine.Engine [generated in 0.00371s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
(2, 'spongebob', 'Spongebob Squarepants')
2023-10-20 00:14:40,039 INFO sqlalchemy.engine.Engine ROLLBACK


In [38]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [40]:
print(select(user_table.c.name, user_table.c.fullname))
print(select(user_table.c["name", "fullname"]))

SELECT user_account.name, user_account.fullname 
FROM user_account
SELECT user_account.name, user_account.fullname 
FROM user_account


In [ ]:
stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

In [43]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

In [44]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [46]:
sandy = User(name="sandy", fullname="Sandy Cheeks")

In [45]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
